# __Reviewer feedback analysis 3__

- Country topical enrichment statistics

## ___Set up___

In [147]:
# For topic enrichment per country p-value multiple testing correction
from statsmodels.stats.multitest import multipletests

# For umap
import pickle, umap
import numpy as np
import pandas as pd
import umap.plot
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib import colors

# for animated gif
import imageio

# for movie
import cv2

from tqdm import tqdm

proj_dir = Path.home() / "projects/plant_sci_hist"

## ___Statistical significance of topical enrichment___

Reviewer 1:

"For Figure 5F it is not clear which of the changes are statistically significant, particularly if controlling for testing of multiple hypotheses."

Response: We have conduct statistical tests on these but did not report the results. They are now incorporated and we modified the figure to indicate significant differences after multiple testing corrections.


### Multiple testing correction

In [10]:
# stat is already generated, but not multiple-testing corrected

# Get the p-values
dir75     = proj_dir / "7_countries/7_5_country_over_time"
pval_file = dir75 / "country_top10_toc_pval.csv"

pval_df = pd.read_csv(pval_file, index_col=[0])
pval_df.head(2)

,toc_name,CHN,USA,JPN,DEU,FRA,GBR,IND,ESP,ITA,AUS
toc,,,,,,,,,,,
0,allergen | pollen | ige | allergenic,4.535153e-42,1.609574e-17,0.885126,1.095708e-12,0.026466,0.000007,7.769514e-01,1.318535e-47,6.467812e-08,0.321514
1,medium | callus | regeneration | culture | som...,3.583564e-38,1.828608e-05,0.538833,4.987797e-18,0.209628,0.018076,5.853805e-186,6.176955e-03,7.353872e-03,0.000003


In [32]:
countries = pval_df.columns[1:]
countries

Index(['CHN', 'USA', 'JPN', 'DEU', 'FRA', 'GBR', 'IND', 'ESP', 'ITA', 'AUS'], dtype='object')

In [33]:
# correct for each country
pval_corrected = []
for country in countries:
  pval = pval_df[country].values
  pval_corrected.append(multipletests(pval, method='fdr_bh')[1])

len(pval_corrected)

10

In [34]:
# Turn into np array than transpose
pval_corrected_arr = np.array(pval_corrected).T

# Convert to dataframe
pval_corrected_df = pd.DataFrame(pval_corrected_arr, columns=list(countries),
                                 index=pval_df.index)
print(pval_corrected_df.shape)

# output to csv
pval_corrected_df.to_csv(dir75 / "country_top10_toc_pval_corrected.csv")

(90, 10)


### Get corrected p-values for Fig 5F

In [35]:
# country order
c_order = ["JPN", "GBR", "DEU", "FRA", "USA", "AUS", "ESP", "ITA", "CHN", "IND"]

# topic order
t_order = [1, 69, 30, 9, 23, 28, 75, 86, 83, 21, 0, 54, 51]

In [42]:
# [[pval_country1], [pval_country2], ...]]
pvals_list = []
for c in c_order:
  pvals = []
  for t in t_order:
    pvals.append(pval_corrected_df.loc[t, c])
  pvals_list.append(pvals)


In [44]:
# Convert to np array
pvals_arr = np.array(pvals_list).T

# Convert to dataframe
pvals_df = pd.DataFrame(pvals_arr, columns=c_order, index=t_order)

# output to csv
pvals_df.to_csv(dir75 / "country_top10_toc_pval_corrected_Fig5F.csv")

In [45]:
pvals_df.head(5)

,JPN,GBR,DEU,FRA,USA,AUS,ESP,ITA,CHN,IND
1,5.638950e-01,2.324108e-02,1.603221e-17,2.482440e-01,2.420217e-05,8.656122e-06,1.010774e-02,1.121777e-02,7.167128e-38,2.634212e-184
69,5.984807e-152,1.680876e-73,1.956934e-30,2.080004e-24,1.110435e-218,2.532725e-22,2.692763e-36,8.805576e-01,3.044451e-203,6.775499e-01
30,1.693636e-40,8.094176e-02,1.119947e-01,7.084776e-02,4.823607e-07,2.079011e-01,2.241177e-01,2.950271e-02,2.599642e-09,1.119393e-01
9,1.079546e-14,7.847829e-10,1.054466e-08,4.100327e-06,2.153816e-20,9.293587e-01,3.920159e-02,5.861011e-01,7.847940e-162,1.830068e-07
23,1.810340e-39,4.011821e-37,2.889594e-43,1.463937e-12,1.889890e-62,2.058462e-08,6.329757e-18,7.114919e-07,0.000000e+00,1.602068e-21
